In [ ]:
from datetime import datetime

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import numpy as np

import efd_utils
import df_utils

In [ ]:
def get_format(istr):
    if ":" in istr:
        return "%Y-%m-%d %H:%M"
    else:
        return "%Y-%m-%d"

In [ ]:
async def get_plot(location, csc_name, topic_name, start_str, end_str):
    start_format = get_format(start_str)
    end_format = get_format(end_str)
 
    start = datetime.strptime(start_str, start_format)
    end = datetime.strptime(end_str, end_format)

    client = efd_utils.get_client(location)
    csc = efd_utils.CSC.from_entry(csc_name)

    query = efd_utils.get_base_query(columns=["private_sndStamp"],
                                     csc_name=csc.name,
                                     csc_index=csc.index,
                                     topic_name=topic_name)
    query += " WHERE " + efd_utils.get_time_clause(date_range=(start, end))
    #print(query)
    ts_df = await client.query(query)

    return hv.Scatter((ts_df.index.values, ts_df.private_sndStamp.values),
                      kdims=hv.Dimension('index'),
                      vdims=hv.Dimension('timestamp', label='private_sndStamp', unit='s'),
                      label=csc.name).opts(width=600)

In [ ]:
start_str = "2019-10-24 12:00"
end_str = "2019-10-24 18:00"

In [ ]:
aths_hb = await get_plot("tucson", "ATHeaderService=0", "logevent_heartbeat", start_str, end_str)
atcam_wb = await get_plot("tucson", "ATCamera=0", "wreb", start_str, end_str)
scrqu_hb = await get_plot("tucson", "ScriptQueue=0", "logevent_heartbeat", start_str, end_str)

In [ ]:
ov = (aths_hb * atcam_wb * scrqu_hb)
ov.opts(legend_position='top_left', click_policy='hide')